In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt 
from torchvision import import datasets, transform

In [ ]:
device = "cuda"  if torch.cuda.is_available() else "cpu"

In [ ]:
transform = transform.ToTensor()
train_data = datasets.MNIST("./data", train=True, download=True,transform=transform)
test_data = datasets.MNIST("./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
model = nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_losses = []
test_accuracies = []

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x, y in train_loader:
        x,y = x.to(device), y.to(device)
        x = x.view(x.size(0), -1)

        preds = model(x)
        loss = criterion(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.items()

    train_losses.append(epoch_loss)

In [ ]:
model.eval()
correct, total = 0,0
with torch.no_grad():
    for x,y in test_loader:
        x,y = x.to(device), y.to(device)
        x = x.view(x.size(0), -1)
        preds = model(x)
        predicted = torch.argmax(preds, dim=1)
        correct += (predicted == y).sum().item()
        total += y.size(0)
acc = correct / total
test_accuracies.append(acc)
print(f"Epoch {epoch+1}: Loss={epoch_loss:.3f}, Accuracy={acc*100:.2f}%")

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(train_losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(1,2,2)
plt.plot(test_accuracies)
plt.title("Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.show()